In [2]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [8]:
# Need a selenium webdriver installed, and then to indicate the location of the driver 
# chrome driver available here: http://chromedriver.chromium.org/downloads
# be sure to unzip it after downloading!

path_to_chromedriver = "C:/Users/Andrew/Downloads/chromedriver_win32/chromedriver.exe"

driver = webdriver.Chrome(path_to_chromedriver)

In [36]:
driver.get("http://www.covenantofmayors.eu/plans-and-actions/action-plans.html")
#let the page load
time.sleep(8)

In [16]:
rows = driver.find_elements_by_xpath('//*[@id="loadItems"]/tr')

In [19]:
first_row = rows[0]

In [27]:
link = first_row.find_element_by_tag_name('a')

In [34]:
link_we_want = link.get_attribute('href')

In [35]:
driver.get(link_we_want)

In [ ]:
for row in rows:
    driver.get(row[0])

In [ ]:
//*[@id="loadItems"]/tr[1]/td[2]

In [37]:
tdry = driver.find_element_by_xpath('//*[@id="tabled-list"]/div[1]/div/div/div')

In [38]:
tdry.click()

In [39]:
ok = driver.find_element_by_xpath('//*[@id="mCSB_11"]')
ARROW_DOWN = u'\ue015'

biglist = False

while not biglist:
    driver.execute_script("window.scrollBy(0,2)");
    ok.send_keys(ARROW_DOWN)
    try:
        itemsper = driver.find_element_by_xpath('//*[@id="mCSB_11_container"]/li[text()="All"]')
        itemsper.click()
#     except ElementNotVisibleException:
#         pass
        biglist = True
    except:
        pass
        

In [40]:
time.sleep(5)
items = driver.find_element_by_id('loadItems')

In [41]:
signatory_table = items.find_elements_by_tag_name('tr')

In [11]:
print("Commitments to be scraped: {}".format(len(signatory_table)))
rowsdata = []
count = 1

for row in signatory_table:
    
    rowdata = row.text.split("\n")
    if len(rowdata) > 3:
        rowsdata.append([rowdata[0], rowdata[1:-1], rowdata[-1]])
    else:
        rowsdata.append(rowdata)
    
    if count % 500 == 0:
        print('At item {}'.format(count))
    count += 1

Commitments to be scraped: 6038
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000


In [12]:
eucov = pd.concat([pd.DataFrame(data = [[i[0], i[1], i[2]]], 
              columns = ['name_and_population', 'commitments', 'signedup_and_actionplan']) for i in rowsdata])
eucov = eucov.reset_index(drop=True)

In [31]:
population = eucov['name_and_population'].str.split('^\D+', expand=True).loc[:, [1]]
names = eucov['name_and_population'].str.split(' \d', expand=True).loc[:, [0]]

In [36]:
eucov['name'] = names
eucov['population'] = population

In [38]:
eucov_out = eucov.loc[: ,['name', 'population', 'commitments']]

In [44]:
eucov_out.to_csv('../../../../output/actors/EU Covenant/population_and_commitment_years.csv')